### Exemplo do site: https://machinelearningmastery.com/understanding-stateful-lstm-recurrent-neural-networks-python-keras/

In [207]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils import np_utils

In [208]:
# fix random seed for reproducibility
numpy.random.seed(7)

In [209]:
# define the raw dataset
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
# create mapping of characters to integers (0-25) and the reverse
char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))

In [210]:
char_to_int

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'K': 10,
 'L': 11,
 'M': 12,
 'N': 13,
 'O': 14,
 'P': 15,
 'Q': 16,
 'R': 17,
 'S': 18,
 'T': 19,
 'U': 20,
 'V': 21,
 'W': 22,
 'X': 23,
 'Y': 24,
 'Z': 25}

In [211]:
int_to_char

{0: 'A',
 1: 'B',
 2: 'C',
 3: 'D',
 4: 'E',
 5: 'F',
 6: 'G',
 7: 'H',
 8: 'I',
 9: 'J',
 10: 'K',
 11: 'L',
 12: 'M',
 13: 'N',
 14: 'O',
 15: 'P',
 16: 'Q',
 17: 'R',
 18: 'S',
 19: 'T',
 20: 'U',
 21: 'V',
 22: 'W',
 23: 'X',
 24: 'Y',
 25: 'Z'}

In [220]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 3
dataX = []
dataY = []
for i in range(0, len(alphabet) - seq_length, 1):
	seq_in = alphabet[i:i + seq_length]
	seq_out = alphabet[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
	print(seq_in, '->', seq_out)

ABC -> D
BCD -> E
CDE -> F
DEF -> G
EFG -> H
FGH -> I
GHI -> J
HIJ -> K
IJK -> L
JKL -> M
KLM -> N
LMN -> O
MNO -> P
NOP -> Q
OPQ -> R
PQR -> S
QRS -> T
RST -> U
STU -> V
TUV -> W
UVW -> X
VWX -> Y
WXY -> Z


In [221]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (len(dataX), seq_length, 1))

In [222]:
X.shape

(23, 3, 1)

In [223]:
# normalize
X = X / float(len(alphabet))

In [224]:
X

array([[[ 0.        ],
        [ 0.03846154],
        [ 0.07692308]],

       [[ 0.03846154],
        [ 0.07692308],
        [ 0.11538462]],

       [[ 0.07692308],
        [ 0.11538462],
        [ 0.15384615]],

       [[ 0.11538462],
        [ 0.15384615],
        [ 0.19230769]],

       [[ 0.15384615],
        [ 0.19230769],
        [ 0.23076923]],

       [[ 0.19230769],
        [ 0.23076923],
        [ 0.26923077]],

       [[ 0.23076923],
        [ 0.26923077],
        [ 0.30769231]],

       [[ 0.26923077],
        [ 0.30769231],
        [ 0.34615385]],

       [[ 0.30769231],
        [ 0.34615385],
        [ 0.38461538]],

       [[ 0.34615385],
        [ 0.38461538],
        [ 0.42307692]],

       [[ 0.38461538],
        [ 0.42307692],
        [ 0.46153846]],

       [[ 0.42307692],
        [ 0.46153846],
        [ 0.5       ]],

       [[ 0.46153846],
        [ 0.5       ],
        [ 0.53846154]],

       [[ 0.5       ],
        [ 0.53846154],
        [ 0.57692308]],

      

In [225]:
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [226]:
y

array([[ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [261]:
# create and fit the model
model = Sequential()
model.add(LSTM(32, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [262]:
#model.fit(X, y, epochs=100, batch_size=1, verbose=2)
model.fit(X, y, epochs=500, batch_size=len(dataX), verbose=2, shuffle=False)

Epoch 1/500
0s - loss: 3.2603 - acc: 0.0000e+00
Epoch 2/500
0s - loss: 3.2588 - acc: 0.0000e+00
Epoch 3/500
0s - loss: 3.2573 - acc: 0.0435
Epoch 4/500
0s - loss: 3.2559 - acc: 0.0435
Epoch 5/500
0s - loss: 3.2544 - acc: 0.0435
Epoch 6/500
0s - loss: 3.2530 - acc: 0.0435
Epoch 7/500
0s - loss: 3.2516 - acc: 0.0435
Epoch 8/500
0s - loss: 3.2501 - acc: 0.0435
Epoch 9/500
0s - loss: 3.2487 - acc: 0.0435
Epoch 10/500
0s - loss: 3.2473 - acc: 0.0435
Epoch 11/500
0s - loss: 3.2459 - acc: 0.0435
Epoch 12/500
0s - loss: 3.2445 - acc: 0.0435
Epoch 13/500
0s - loss: 3.2431 - acc: 0.0435
Epoch 14/500
0s - loss: 3.2417 - acc: 0.0435
Epoch 15/500
0s - loss: 3.2403 - acc: 0.0435
Epoch 16/500
0s - loss: 3.2388 - acc: 0.0435
Epoch 17/500
0s - loss: 3.2374 - acc: 0.0435
Epoch 18/500
0s - loss: 3.2360 - acc: 0.0435
Epoch 19/500
0s - loss: 3.2346 - acc: 0.0435
Epoch 20/500
0s - loss: 3.2331 - acc: 0.0435
Epoch 21/500
0s - loss: 3.2317 - acc: 0.0435
Epoch 22/500
0s - loss: 3.2302 - acc: 0.0435
Epoch 23/50

In [263]:
# summarize performance of the model
scores = model.evaluate(X, y, verbose=0)
print("Model Accuracy: %.2f%%" % (scores[1]*100))

Model Accuracy: 100.00%


In [264]:
# demonstrate some model predictions
for pattern in dataX:
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(len(alphabet))
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	print(seq_in, "->", result)

['A', 'B', 'C'] -> D
['B', 'C', 'D'] -> E
['C', 'D', 'E'] -> F
['D', 'E', 'F'] -> G
['E', 'F', 'G'] -> H
['F', 'G', 'H'] -> I
['G', 'H', 'I'] -> J
['H', 'I', 'J'] -> K
['I', 'J', 'K'] -> L
['J', 'K', 'L'] -> M
['K', 'L', 'M'] -> N
['L', 'M', 'N'] -> O
['M', 'N', 'O'] -> P
['N', 'O', 'P'] -> Q
['O', 'P', 'Q'] -> R
['P', 'Q', 'R'] -> S
['Q', 'R', 'S'] -> T
['R', 'S', 'T'] -> U
['S', 'T', 'U'] -> V
['T', 'U', 'V'] -> W
['U', 'V', 'W'] -> X
['V', 'W', 'X'] -> Y
['W', 'X', 'Y'] -> Z


In [313]:
palavra = 'CBA'

In [314]:
intLetra = [char_to_int[letra] for letra in palavra]

In [315]:
intLetra

[2, 1, 0]

In [316]:
x = numpy.reshape(intLetra, (1, len(intLetra), 1))

In [317]:
x = x / float(len(alphabet))

In [318]:
x

array([[[ 0.07692308],
        [ 0.03846154],
        [ 0.        ]]])

In [319]:
prediction = model.predict(x, verbose=0)

In [320]:
prediction

array([[  2.93182056e-05,   1.14010982e-05,   2.44234288e-05,
          2.38689065e-01,   2.80550092e-01,   2.79667497e-01,
          1.60549134e-01,   3.16995159e-02,   6.16291445e-03,
          1.49015780e-03,   7.56011868e-04,   1.23154066e-04,
          1.56743365e-04,   4.42606870e-05,   1.70980766e-05,
          1.00968555e-05,   1.18734715e-05,   2.75170828e-06,
          3.41576583e-06,   7.78987612e-07,   1.00041305e-07,
          5.27512576e-08,   3.45342670e-08,   1.35623024e-08,
          6.40884679e-09,   9.91802107e-09]], dtype=float32)

In [321]:
index = numpy.argmax(prediction)

In [322]:
index

4

In [323]:
result = int_to_char[index]

In [324]:
result

'E'